## Repeat with last year's data to compare with latest year

### Load data for last year

In [ ]:
import pandas as pd

# We use Jan 2020 data
trade_lastyr = pd.read_excel("export_2020m1.xlsx", skiprows=1, index_col=0, nrows=7)
trade_lastyr

### Create network

In [ ]:
# We first convert the matrix data into pair data
trade_lastyr = pd.melt(trade_lastyr.reset_index(), id_vars='index')
trade_lastyr.columns = ['export_country', 'import_country', 'value']
trade_lastyr.head()

In [ ]:
trade_lastyr.dropna(inplace=True)
trade_lastyr.head()

In [ ]:
# Now we can use the pairs to create the graph, 
# with the first two columns representing the nodes,
# and the last column representing the weight

import networkx as nx
import matplotlib.pyplot as plt

G_lastyear = nx.Graph()

for index, row in trade_lastyr.iterrows():
    G_lastyear.add_edge(row['export_country'], row['import_country'], weight=row['value'])

# remove isolated vertices (if any)    
remove = [node for node,degree in G_lastyear.degree() if degree ==0]
G_lastyear.remove_nodes_from(remove)

#Setting size and colours
options = {
     'node_color': 'lightblue',
     'edge_color': 'lightblue',
     'node_size': 1,
     'width': 1,
     'alpha': 1.0}

### Load back latest year's network graph

In [ ]:
#load back latest year's network graph
import pickle

f = open("graph_pickle_latest", "rb")
G_latest = pickle.load(f)
f.close()

### Draw graphs to compare output from last year with latest year using a 3x2 subplot containing all the network representations

In [ ]:
fig, ax = plt.subplots(3, 2, figsize = (10, 15))
layouts = {"spring": nx.spring_layout, "random": nx.random_layout, "circular": nx.circular_layout}
graphs = {"latest": G_latest, "last year's": G_lastyear}

for i, (layout_name, layout) in enumerate(layouts.items()):  
    for j, (graph_name, graph) in enumerate(graphs.items()):
        pos = layout(graph)
        nx.draw_networkx(graph, pos=pos, ax=ax[i][j], font_size=9,**options)
        plt.tight_layout()
        ax[i][j].set_axis_off()
        ax[i][j].set_title(f"{layout_name} layout for {graph_name} data")